# NRC-09: Darks FPA and ASIC Temperature Plots

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Plot FPA and ASIC temperatures](#fpa)
* [Check temperature transients and affect on the data ](#temp)
* [Compare with other data](#compare)

<a id='objective'></a>
## Objective

We need to double check that focal plane temperatures and DC levels are what we've seen at other times. Generate plots of the FPA and ASIC temperatures and see if they are well-behaved, and see what the consequence is on the data if there are any temperature transients. Compare with ground test data. 

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used:
```python
pip install ipython jupyter matplotlib pylint pandas
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.time import Time
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator
%matplotlib inline

In [ ]:
def open_w_pandas(path, data_label=None):
    """
    Read a .csv file downloaded from the JWST EDB into a pandas DataFrame.
    
    :param path:  A .csv file with data from the JWST EDB.
    :type path:  str
    
    :param data_label:  An optional substitute label to overwrite the data column label.
    :type data_label:  str
    """
    
    # Use the pandas built-in function to read the provided .csv file.
    pd_frame = pd.read_csv(path)
    
    # Drop an empty unnecessary column from the DataFrame.
    if 'Unnamed: 4' in pd_frame.columns:
        pd_frame.drop(columns=['Unnamed: 4'], inplace=True)
    
    # If the substitute label is provided, replace the 'euvalue' column label.
    if data_label:
        pd_frame.rename(index=str, columns={"euvalue": data_label}, inplace=True)
        
    return pd_frame

In [ ]:
def extract_from_engdb(input_hdulist, url_base, config):
    """Provide a link to the EngDB and a config file containing the desired
       mnemonics to extract values for headers to be added to the output file.
       e.g.
       http://iwjwdmsbemweb.stsci.edu/JWDMSEngSpAccB71/TlmMnemonicDataSrv.svc/MetaData/TlmMnemonics/
    
    Parameters
    ----------
    input_hdulist : FITS HDUList
        Old exposure headers
    url_base : string
        Engineering database URL
    config : string
        YAML format configuration file that includes "mnemonics" section
    
    Returns
    -------
    input_hdulist : FITS HDUList
        Updated FITS HDUList with mnemonic headers added
    """

    # Get the exposure start and end times
    start_time = Time(input_hdulist[0].header['DATE-OBS']+'T'+old_hdulist[0].header['TIME-OBS'], format='fits', scale='utc').isot
    end_time = Time(input_hdulist[0].header['DATE-OBS']+'T'+old_hdulist[0].header['TIME-END'], format='fits', scale='utc').isot
    params = {'sTime':start_time, 'eTime':end_time}

    # Start HTTP request session
    session = requests.Session()

    for keyword, mnemonic in config['mnemonics'].items():

        # Get request to server.
        url = url_base + mnemonic
        req = session.get(url, params=params, verify=False)

        # Parse json
        parsed_json = req.json()

        # json ObsTime has format '/Date(1358619814230+0000)/' which is 1358619814.230 in UNIX time
        # isotime = Time(float(parsed_json['Data'][0]['ObsTime'][6:-7])/1000., format='unix').isot

        # Take the first value of the series (there are no values right now,
        # so use EUType instead of EUValue just to test functionality)
        # new_hdulist[0].header[keyword] = (parsed_json['TlmMnemonics'][0]['EUValue'], mnemonic.upper())
        input_hdulist[0].header[keyword] = (parsed_json['TlmMnemonics'][0]['EUType'], mnemonic.upper())

    # Add the Engineering Mnemonics section heading
    input_hdulist[0].header.set('', '', before=list(config['mnemonics'].keys())[0])
    input_hdulist[0].header.set('', 'Engineering Mnemonics', before=list(config['mnemonics'].keys())[0])
    input_hdulist[0].header.set('', '', before=list(config['mnemonics'].keys())[0])

    return input_hdulist

<a id='data'></a>
## Data for analysis

Observation data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
files = glob(base_dir+ground+'/NRC*_484_*uncal.fits')

Telemetry can either be retrieved from the JWST Engineering Database or downloaded into a CSV file, e.g.,: 

In [ ]:
OBA_MODA_TEMPERATURES = "IGDP_NRC_A_T_OBA_TS1-20190531T235959-20190601T235959.csv"
OBA_MODB_TEMPERATURES = "IGDP_NRC_B_T_OBA_TS1-20190531T235959-20190601T235959.csv"
FPA_MODA_SW_TEMPERATURES = "IGDP_NRC_FA_TMC2_SWMV_T-20190531T235959-20190601T235959.csv"
FPA_MODB_SW_TEMPERATURES = "IGDP_NRC_FB_TMC2_SWMV_T-20190531T235959-20190601T235959.csv"
FPA_MODA_LW_TEMPERATURES = "IGDP_NRC_FA_TMC2_LWMV_T-20190531T235959-20190601T235959.csv"
FPA_MODB_LW_TEMPERATURES = "IGDP_NRC_FB_TMC2_LWMV_T-20190531T235959-20190601T235959.csv"

<a id='fpa'></a>
## Plot FPA and ASIC temperatures

Check that temperatures are well-behaved. 

Start with downloaded data provided above. One example of mnemonics are the module A and B SW and LW FPA temperature measurements. These measurements are contained in the downloaded CSV files. We'll start with a set of .csv files for these data sets over the same 24-hour period.

In [ ]:
# Information for OBA temperatures
X_FRAME_OBA = open_w_pandas(OBA_MODA_TEMPERATURES, data_label="x_values")
Y_FRAME_OBA = open_w_pandas(OBA_MODB_TEMPERATURES, data_label="y_values")
print(X_FRAME_OBA.shape)
print(X_FRAME_OBA.columns)

In [ ]:
# OBA temperature plots 
plt.figure(); X_FRAME_OBA.plot(x='MJD', y='x_values', kind='scatter'); Y_FRAME_OBA.plot(x='MJD', y='y_values', kind='scatter');

In [ ]:
# Information for FPA SW temps
X_FRAME_FPA_SW_A = open_w_pandas(FPA_MODA_SW_TEMPERATURES, data_label="x_values")
Y_FRAME_FPA_SW_B = open_w_pandas(FPA_MODB_SW_TEMPERATURES, data_label="y_values")
print(X_FRAME_FPA_SW_A.shape)
print(X_FRAME_FPA_SW_A.columns)

In [ ]:
# FPA SW temperature plots 
plt.figure(); X_FRAME_FPA_SW_A.plot(x='MJD', y='x_values', kind='scatter'); Y_FRAME_FPA_SW_B.plot(x='MJD', y='y_values', kind='scatter');

In [ ]:
# Information for FPA LW temps
X_FRAME_FPA_LW_A = open_w_pandas(FPA_MODA_LW_TEMPERATURES, data_label="x_values")
Y_FRAME_FPA_LW_B = open_w_pandas(FPA_MODB_LW_TEMPERATURES, data_label="y_values")
print(X_FRAME_FPA_LW_A.shape)
print(X_FRAME_FPA_LW_A.columns)

In [ ]:
# FPA LW temperature plots 
plt.figure(); X_FRAME_FPA_LW_A.plot(x='MJD', y='x_values', kind='scatter'); Y_FRAME_FPA_LW_B.plot(x='MJD', y='y_values', kind='scatter');

<a id='temp'></a>
## Check temperature transients and affect on the data 

Take a look at the pixel values for the files.

|__SW dark current (1000s)__         | __LW dark current (1000s)__   |
|:---                   |:---              |
| 1.9 ± 1.1 e–          | 27 ± 5 e–        |


If the data doesn't contain FPA or ASIC temperature headers, we can start by searching the Engineering database for the mnemonics we want (FPA and ASIC temperatures) during the time of the observations, and add this information to the FITS files in the "Engineering Mnemonics" section of the headers. 

In [ ]:
for file in output_files: 
    with fits.open(file) as input_hdulist:
        updated_list = extract_from_engdb(input_hdulist, url_base, config)
        updated_list.writeto(file, overwrite=True)

### Run scripts to get statistics 

Next, run the script ```dark_stats_with_temps.py```. This will output a series of ```.dat``` tables into the current working directory (or another that is specified). We will use the tables in the plots below. 

In [ ]:
# !python dark_stats_with_temps.py './'

Now that you have the data tables, generate the plots. 

### Generate mean pixel values for each group using temperature as the colormap

In [ ]:
# This section is for testing scripts and will not be needed once we have flight data 
detectors = {
            '481': 'NRCA1',
            '482': 'NRCA2',
            '483': 'NRCA3',
            '484': 'NRCA4',
            '485': 'NRCA5',
            '486': 'NRCB1',
            '487': 'NRCB2',
            '488': 'NRCB3',
            '489': 'NRCB4',
            '490': 'NRCB5'
            }

In [ ]:
base = "flight_data/calc_dark_current"

for k,v in sorted(detectors.items()):

    df = ascii.read(base+k+".dat")
    subarrays = np.unique(df['subarray'])
    
    for sub in subarrays:
        findf = df[df['subarray'] == sub]
        fig = plt.figure(figsize=(10,7))
        points = plt.scatter(findf["group"], findf["mean"],
                             c=findf["asic_temp"], s=75, cmap="viridis")
        cbar = plt.colorbar(points)
        cbar.set_label('ASIC temperature (K)',fontsize=13)

        plt.xlabel('Frame',fontsize=13)
        plt.ylabel('3-$\sigma$ clipped mean signal (ADU)',fontsize=13)
        plt.title(detectors[k]+"\nsubarray "+sub,fontsize=13)
#         plt.savefig(outname+"_subarray"+sub+".png")

<a id='compare'></a>
## Compare with other data

Repeat the exercise above using ground or other flight data. 

Next, run the script ```dark_stats_with_temps.py```. This will output a series of ```.dat``` tables into the current working directory (or another that is specified). We will use the tables in the plots below. 

In [ ]:
# !python dark_stats_with_temps.py './'

Now that you have the data tables, generate the plots. 

In [ ]:
base = "otis_long_darks/calc_dark_current"

for k,v in sorted(detectors.items()):

    df = ascii.read(base+k+".dat")
    subarrays = np.unique(df['subarray'])
    
    for sub in subarrays:
        findf = df[df['subarray'] == sub]
        fig = plt.figure(figsize=(10,7))
        points = plt.scatter(findf["group"], findf["mean"],
                             c=findf["asic_temp"], s=75, cmap="viridis")
        cbar = plt.colorbar(points)
        cbar.set_label('ASIC temperature (K)',fontsize=13)

        plt.xlabel('Frame',fontsize=13)
        plt.ylabel('3-$\sigma$ clipped mean signal (ADU)',fontsize=13)
        plt.title(detectors[k]+"\nsubarray "+sub,fontsize=13)
#         plt.savefig(outname+"_subarray"+sub+".png")